In [ ]:
import numpy as np
import os

In [ ]:
target_algos = ['yin', 'maddox', 'noll', 'hermes', 'crepe', 'boersma']
data_dir = '/Akamai/voice/data/pitches-crepe-assisted/'
vuv_dir = '/Akamai/voice/data/pitches-raw-crepe-assisted/'
res_dir = '/Akamai/voice/data/pitches-vuv-crepe-assisted/'

In [ ]:
min_f0 = 60
max_f0 = 2000

In [ ]:
def separate(adir):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

def apply_detection (uy, ay):
    uy = uy[:ay.size]
    ay[uy == 0] = 0
    return ay
    
def save_song (path, x, y):
    fout = open(path, "w+")
    for i in range(x.size):
        fout.write(f"{x[i]} {y[i]}\n")
    fout.close()

ignore_files = ['raw.txt', 'bass', 'middle', 'top']

def ignore(name):
    for suffix in ignore_files:
        if suffix in location:
            return True
    return False

In [ ]:
def impose_unvoice(cy, ay):
    for i in range(min(len(cy), len(ay))):
        if cy[i] == 0:
            ay[i] = 0.0
    return ay

In [ ]:
crepe_dir = '/Akamai/voice/data/pitches-vuv-new/crepe/'
pitches_dir = '/Akamai/voice/data/pitches-crepe-assisted/'
res_dir = '/Akamai/voice/data/pitches-vuv-crepe-assisted'

algorithms = ['boersma', 'hermes', 'noll', 'yin']

for collection in os.listdir(crepe_dir):
    for song in os.listdir(os.path.join(crepe_dir, collection)):
        for singer in os.listdir(os.path.join(crepe_dir, collection, song)):
            ct, cy = separate(os.path.join(crepe_dir, collection, song, singer))
            for algo in algorithms:
                try:
                    at, ay = separate(os.path.join(pitches_dir, algo, collection, song, singer))
                except OSError:
                    print(f"{algo}-{collection}-{song} doens't exist")
                    continue
                
                new_ay = impose_unvoice(cy, ay)
                save_song(os.path.join(res_dir, algo, collection, song, singer), at, new_ay)